### 1. Nearest Neighbors
- http://scikit-learn.org/stable/modules/neighbors.html
- http://note.youdao.com/noteshare?id=002cc00cdfdd7564e80bda1673354195&sub=9236F7816FCC4AEF8CA9AC4711C1C84C



#### 1.1 Finding the Nearest Neighbors
For the simple task of finding the nearest neighbors between two sets of data, the unsupervised algorithms within sklearn.neighbors can be used:

In [1]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)  # 最近的两个点
distances, indices = nbrs.kneighbors(X)

In [3]:
distances

array([[ 0.        ,  1.        ],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.41421356],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.41421356]])

In [4]:
indices

array([[0, 1],
       [1, 0],
       [2, 1],
       [3, 4],
       [4, 3],
       [5, 4]], dtype=int64)

Because the query set matches the training set, the nearest neighbor of each point is the point itself, at a distance of zero.

It is also possible to efficiently produce a sparse graph showing the connections between neighboring points:

In [5]:
nbrs.kneighbors_graph(X).toarray()  # 距离最近的两个点标记为1，其他点标记为0

array([[ 1.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  1.]])

Our dataset is structured such that points nearby in index order are nearby in parameter space, leading to an approximately block-diagonal matrix of K-nearest neighbors. Such a sparse graph is useful in a variety of circumstances which make use of spatial relationships between points for unsupervised learning: in particular, see `sklearn.manifold.Isomap`, `sklearn.manifold.LocallyLinearEmbedding`, and `sklearn.cluster.SpectralClustering`.

#### 1.2 KDTree and BallTree Classes(不同的计算最近邻点的算法)
Alternatively, one can use the `KDTree` or `BallTree` classes directly to find nearest neighbors. This is the functionality wrapped by the `NearestNeighbors` class used above. 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html#sklearn.neighbors.KDTree
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.BallTree.html#sklearn.neighbors.BallTree
- KDTree中的参数`leaf_size`: Changing `leaf_size` will not affect the results of a query, but can significantly impact the speed of a query and the memory required to store the constructed tree.
- 不同的距离函数：http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html

The Ball Tree and KD Tree have the same interface; we’ll show an example of using the KD Tree here:

In [7]:
from sklearn.neighbors import KDTree
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
kdt = KDTree(X, leaf_size=30, metric='euclidean')
kdt.query(X, k=2, return_distance=False) ## 查询最近的两个点

array([[0, 1],
       [1, 0],
       [2, 1],
       [3, 4],
       [4, 3],
       [5, 4]], dtype=int64)

In [10]:
import numpy as np
import pickle
np.random.seed(0)
X2 = np.random.random((10, 3))  # 10 points in 3 dimensions
tree = KDTree(X2, leaf_size=2)        
s = pickle.dumps(tree)      # 建好的tree可以保存起来               

In [14]:
X2.shape

(10, 3)

In [15]:
tree_copy = pickle.loads(s)    # 加载已经建好的tree            
dist, ind = tree_copy.query(X2[0].reshape(1,-1), k=3)     
print(ind)  # indices of 3 closest neighbors
print(dist)  # distances to 3 closest neighbors

[[0 3 1]]
[[ 0.          0.19662693  0.29473397]]


##### 寻找指定半径内的临近点 (query the tree for neighbors within a radius r)

In [18]:
import numpy as np
np.random.seed(0)
X3 = np.random.random((10, 3))  # 10 points in 3 dimensions
tree = KDTree(X3, leaf_size=2)     
print(tree.query_radius(X3[0].reshape(1,-1), r=0.3, count_only=True))  # 只返回半径范围内的邻居点的个数

ind = tree.query_radius(X3[0].reshape(1,-1), r=0.3)  
print(ind)  # indices of neighbors within distance 0.3

[3]
[array([3, 0, 1], dtype=int64)]


---
如果我使用`RadiusNeighborsRegressor`建立一个预测误差的模型，效果会怎么样？

In [19]:
X = [[0], [1], [2], [3]]
y = [0, 0, 1, 1]
from sklearn.neighbors import RadiusNeighborsRegressor
neigh = RadiusNeighborsRegressor(radius=1.0)
neigh.fit(X, y)

RadiusNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
             metric_params=None, p=2, radius=1.0, weights='uniform')

In [20]:
neigh.predict([[1.5]])

array([ 0.5])